In [ ]:
from Tensor import Tensor
from optim import SGD
import numpy as np

params1 = Tensor(np.random.rand(3,2), requires_grad=False)
params2 = Tensor(np.random.rand(3,2))
optim = SGD([params1, params2])
out = params1 * params2
out = out.mean()
optim.zero_grad()
out.backward()
optim.step()

In [ ]:
import torch
torch.nn.Parameter

In [4]:
from Tensor import Tensor


x = Tensor([5])
b = Tensor([2])
c =  Tensor([2.3])
d = Tensor([-3.4])

for i in range(10):
    x = x + 3
    x.backward()
    x.grad = None
    print("CONTEXT LEN PARENTS :", len(x._ctx.saved_tensors))

CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2
CONTEXT LEN PARENTS : 2


In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch import nn
from torch import Tensor
import torch
import numpy as np

torch.manual_seed(23)
np.random.seed(23)

digits = datasets.load_digits()
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.3, shuffle=False
)

def get_weights(in_dim, out_dim):
    # Xavier/Glorot initialization
    std = np.sqrt(2.0 / (in_dim + out_dim))
    return np.random.normal(0, std, (in_dim, out_dim)).astype(np.float32)


class Linear:
    def __init__(self, in_dim: int, out_dim: int):
        self.in_dim = in_dim
        self.out_dim = out_dim

        weights = get_weights(self.in_dim, self.out_dim)
        self.weight = nn.Parameter(torch.tensor(weights), requires_grad=True)

    def forward(self, x: Tensor):
        return x.matmul(self.weight)


class MLP(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        super().__init__()
        self.l1 = Linear(in_dim, 128)
        self.l2 = Linear(128, out_dim)

        self.l1.weight.retain_grad()
        self.l2.weight.retain_grad()

    def forward(self, x: Tensor):
        x = self.l1.forward(x)
        x = x.relu()
        return self.l2.forward(x)

model = MLP(8 * 8, 10)
lr = 0.001
# sgd = torch.optim.SGD([model.l1.weight, model.l2.weight], lr=lr)
count = 0
# X_train = X_train[:1]
# y_train = y_train[:1]
for epoch in range(1):
    progress_bar = tqdm(zip(X_train, y_train), total=len(y_train), desc="loss: 0.0")
    for sample, target in progress_bar:
        sample = Tensor(sample)

        one_hot_target = np.zeros(10)
        one_hot_target[target] = 1
        target = Tensor(one_hot_target)

        logits = model.forward(sample.reshape(1, -1))
        logits.retain_grad()

        probs = logits.softmax(-1)
        probs.retain_grad()

        safe_probs = probs + 1e-8
        log_probs = safe_probs.log()
        log_probs.retain_grad()
        loss_dot = target.dot(log_probs.reshape(-1))
        loss = -loss_dot


        model.l1.weight.grad = None
        model.l2.weight.grad = None
        model.l1.weight.retain_grad()
        model.l2.weight.retain_grad()

        debug = logits
        debug.retain_grad()
        loss.backward()

        count += 1
        # if count == 1:
        #     print("TYPE: ", debug.dtype)
        #     print("debug var item: ", debug)
        #     print("debug var GRAD: ", debug.grad)
        #     print("LOSS: ", loss)
        #     print("model.l1 GRAD: ", model.l1.weight.grad)
        #     break

        model.l1.weight = model.l1.weight - lr * model.l1.weight.grad
        model.l2.weight = model.l2.weight - lr * model.l2.weight.grad
        progress_bar.set_description_str(f"loss: {float(loss.item()):.4f}")

sample_idx = 2
print("TARGET: ", y_test[sample_idx])

x = Tensor(X_test[sample_idx])
res = model.forward(x)
print(torch.argmax(res.detach()).item())

loss: 0.3683: 100%|██████████| 1257/1257 [00:01<00:00, 653.04it/s]


TARGET:  5
5


In [1]:
import torch
import numpy as np
from Tensor import Tensor

for i in range(5):
    # Generate the same random values
    np.random.seed(i)
    arr = np.random.randn(3, 4).astype(np.float32)

    # Torch tensor
    torch_t = torch.tensor(arr, requires_grad=True)
    torch_softmax = torch_t.softmax(-1)
    torch_loss = torch_softmax.sum()
    torch_loss.backward()
    
    # Custom Tensor
    my_t = Tensor(arr, requires_grad=True)
    my_softmax = my_t.softmax()
    my_loss = my_softmax.sum()
    my_loss.backward()

    print(f"Iteration {i}")
    print("Torch softmax:\n", torch_softmax.detach().numpy())
    print("My softmax:\n", my_softmax.item)
    print("Difference:\n", np.abs(torch_softmax.detach().numpy() - my_softmax.item).max())
    print("Torch grad:\n", torch_t.grad.numpy())
    print("My grad:\n", my_t.grad)
    print("Grad difference:\n", np.abs(torch_t.grad.numpy() - my_t.grad).max())
    print("-" * 40)

Iteration 0
Torch softmax:
 [[0.30096766 0.07694629 0.13723414 0.48485196]
 [0.6287442  0.03655759 0.25120136 0.08349688]
 [0.11495472 0.19216523 0.14720124 0.5456788 ]]
My softmax:
 [[0.30096766 0.0769463  0.13723414 0.48485196]
 [0.6287442  0.03655759 0.25120136 0.08349688]
 [0.11495472 0.19216523 0.14720125 0.5456788 ]]
Difference:
 1.4901161e-08
Torch grad:
 [[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]
 [6.851835e-09 1.145394e-08 8.773878e-09 3.252499e-08]]
My grad:
 [[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-7.4952148e-08 -4.3580037e-09 -2.9945536e-08 -9.9536033e-09]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]
Grad difference:
 7.495215e-08
----------------------------------------
Iteration 1
Torch softmax:
 [[0.7749224  0.08281929 0.09003924 0.05221913]
 [0.27410856 0.01154876 0.6604546  0.0538881 ]
 [0.20853183 0.11811778 0.65403473 0.01931558]]
My softmax:
 [[0.7749224  0